# Asyncio Event Loop in Action 

Asynchronous tasks are not parallel. That caveat has to be stated upfront. However everytime when our code reaches some external dependency like response from database or web resource we don't have to hang untill we get our result. In case we have something other to do (and possibly another task that waits to be replied) we can put all those tasks in asyncio event loop. But first let's see the case in synchronous scenario.

In [7]:
import asyncio
import time
from asyncio import sleep as asleep


def process(n):
    global start_sync
    time.sleep(n)
    print(f'done process {n} - {time.time()-start_sync:.1f} seconds from start')
    return


def run_processes():
    process(2)
    process(3)
    process(4)
    return

For now we will ignore global variable and the print statement. Process is basically some function that runs exactly n seconds. This might imitate waiting for some database response. We will also introduce function that runs three processes one by one. This should take 2s + 3s + 4s = 9s. To see how long it takes for each process to end I will let them use global variable `start_sync`. Just before returning every function will print its duration. Example is pretty straight forward:

In [8]:
start_sync = time.time()
run_processes()
end_sync = time.time()

print('SYNC TIME: ', f'{end_sync - start_sync:.1f}')

done process 2 - 2.0 seconds from start
done process 3 - 5.0 seconds from start
done process 4 - 9.0 seconds from start
SYNC TIME:  9.0


Now let's change `process` to `aprocess` which is asynchronous. It also has some sleeping to do, but while it is doing it, await statement lets our program go somewhere else. We will also run these `aprocesses` but we will put them into event loop using `asyncio.gather()` and `asyncio.run()`.

In [9]:
async def aprocess(n):
    global start_async
    await asleep(n)
    print(f'done async process {n} - {time.time()-start_async:.1f} seconds from start')
    return


async def run_aprocesses():
    await asyncio.gather(
        aprocess(2),
        aprocess(3),
        aprocess(4)
    )
    return

And analogously we will report its execution but this time we have to await our gathering function since it is also awaiting for every task to return back to it. Important note here. In jupyter notebook, there is already running event loop, so for now instead of `asyncio.run()` we will just `await`. In other words, we don't have to create new event loop because jupyter started one for us.

In [12]:
start_async = time.time()
await run_aprocesses()  # just await (because of jupyter notebook)
end_async = time.time()

print('ASYNC TIME: ', f'{end_async - start_async:.1f}')

done async process 2 - 2.0 seconds from start
done async process 3 - 3.0 seconds from start
done async process 4 - 4.0 seconds from start
ASYNC TIME:  4.0


So as you can see, `aprocesses` started almost simultaneously (important word is almost). Every one of them encountered await statement and returned back to loop so it could trigger next task and then it was constantly revisiting tasks to see how they are doing. Hence the total asynchronous time is approximately the same as time of the longest task. To run whole example outside of jupyter notebook I will paste the whole example. See the difference in actually using `asyncio.run()` this time.

In [ ]:
import asyncio
import time
from asyncio import sleep as asleep


def process(n):
    global start_sync
    time.sleep(n)
    print(f'done process {n} - {time.time()-start_sync:.1f} seconds from start')
    return


def run_processes():
    process(2)
    process(3)
    process(4)
    return


async def aprocess(n):
    global start_async
    await asleep(n)
    print(f'done async process {n} - {time.time()-start_async:.1f} seconds from start')
    return


async def run_aprocesses():
    await asyncio.gather(
        aprocess(2),
        aprocess(3),
        aprocess(4)
    )
    return


if __name__ == '__main__':
    start_sync = time.time()
    run_processes()
    end_sync = time.time()

    print('SYNC TIME: ', f'{end_sync - start_sync:.1f}')

    start_async = time.time()
    asyncio.run(run_aprocesses())
    end_async = time.time()

    print('ASYNC TIME: ', f'{end_async - start_async:.1f}')

In next example let's see how we can look into event loop. Let's create global logging dictionary called `LOGGER` and global counter called `RUNNING`. We will use this variables to let every task write to it.

In [23]:
LOGGER = {}
RUNNING = 0


async def some_work(name, t):
    global LOGGER, RUNNING
    start = time.time()
    LOGGER.update({name: 'ONGOING'})
    RUNNING += 1
    await asleep(t)
    LOGGER.update({name: f'duration: {time.time()-start:.2f}'})
    RUNNING -= 1
    return

Here make note that we have sandwitched awaitable action so it reports its timings to global variables. Now we will add another task that will report back to us, what is happening in the event loop every 2 seconds.

In [24]:
async def read_statuses():
    start_async = time.time()
    while True:
        print(f'uptime: {time.time() - start_async:.2f}')
        print(f'Events in the loop: {LOGGER}')
        print(f'Still running: {RUNNING}\n')

        if not RUNNING: # important to end this coroutine after everything else returned
            break

        await asleep(2)

Now we will run everything in one event loop.

In [22]:
tasks_to_complete = [some_work('cleaning', 3),
                     some_work('reading', 7),
                     some_work('writing', 14)]
tasks_to_complete.extend([read_statuses()]) # remember to add our reader to task list!

loop = asyncio.new_event_loop()
asyncio.set_event_loop(loop)  # Python 3.10+ specific
loop.run_until_complete(asyncio.gather(*tasks_to_complete))
loop.close()

This one might not work in jupyter notebook so I will paste you what to expect in any other environment:

uptime: 0.00
Events in the loop: {'cleaning': 'ONGOING', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 3


uptime: 2.00
Events in the loop: {'cleaning': 'ONGOING', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 3


uptime: 4.00
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 2


uptime: 6.00
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'ONGOING', 'writing': 'ONGOING'}
Still running: 2


uptime: 8.01
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'duration: 7.00', 'writing': 'ONGOING'}
Still running: 1


uptime: 10.01
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'duration: 7.00', 'writing': 'ONGOING'}
Still running: 1


uptime: 12.01
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'duration: 7.00', 'writing': 'ONGOING'}
Still running: 1


uptime: 14.01
Events in the loop: {'cleaning': 'duration: 3.00', 'reading': 'duration: 7.00', 'writing': 'duration: 14.00'}
Still running: 0

As you can see, we started three tasks and every two seconds we were reading the current status of event loop contents.